In [ ]:
%loadFromPOM pom.xml

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [ ]:
// Find a patient whose given name is 大明 and family name is 陳
Bundle results = client
    .search()
    .forResource(Patient.class)
    .where(Patient.GIVEN.matches().value("大明"))
    .and(Patient.FAMILY.matches().value("陳"))
    .returnBundle(Bundle.class)
    .execute();

// Get the first patient from the bundle
Patient patient = (Patient) results.getEntry().get(0).getResource();

In [ ]:
// find the practitioner whose family name is  Kelly Smith
Bundle results = client
    .search()
    .forResource(Practitioner.class)
    .where(Practitioner.NAME.matches().value("Kelly Smith"))
    .returnBundle(Bundle.class)
    .execute();

// Get the first practitioner from the bundle
Practitioner practitioner = (Practitioner) results.getEntry().get(0).getResource();


In [ ]:
// search for the patient's appointments
results = client
    .search()
    .forResource(Appointment.class)
    .where(Appointment.PATIENT.hasId(patient.getIdElement().getIdPart()))
    .returnBundle(Bundle.class)
    .execute();

// Get the first appointment from the bundle
Appointment appointment = (Appointment) results.getEntry().get(0).getResource();


In [ ]:
// create Condition for hypertension. Hypertension is encoded as 38341003 in http://snomed.info/sct system. with display text "Hypertension"
Condition condition = new Condition();
condition.setSubject(new Reference("Patient/" + patient.getIdElement().getIdPart()));
condition.setCode(new CodeableConcept().addCoding(new Coding("http://snomed.info/sct", "38341003", "Hypertension")));

//save the condition to the server
MethodOutcome outcome = client.create().resource(condition).execute();

// get the id of the condition
String conditionId = outcome.getId().getIdPart();


In [ ]:
// create an encounter
Encounter encounter = new Encounter();
encounter.setSubject(new Reference("Patient/" + patient.getIdElement().getIdPart()));
encounter.addDiagnosis(new Encounter.DiagnosisComponent(new Reference("Condition/" + conditionId)));
encounter.addAppointment(new Reference("Appointment/" + appointment.getIdElement().getIdPart()));
encounter.setStatus(Encounter.EncounterStatus.INPROGRESS);

// set participant type to "primary performer"
Encounter.EncounterParticipantComponent participant = new Encounter.EncounterParticipantComponent();
participant.addType().addCoding().setSystem("http://terminology.hl7.org/CodeSystem/v3-ParticipationType").setCode("PPRF");
participant.setIndividual(new Reference("Practitioner/" + practitioner.getIdElement().getIdPart()));
encounter.addParticipant(participant);

// create the encounter on the server
MethodOutcome outcome = client.create().resource(encounter).execute();

// print the outcome
System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(outcome.getResource()));

// get the id of the encounter
String encounterId = outcome.getId().getIdPart();

In [ ]:
// get the encounter from the server
Encounter encounter = client.read().resource(Encounter.class).withId(outcome.getId().getIdPart()).execute();

//create a new observation
// temperature 96.8 F
// blood pressure 130/80
Observation observation = new Observation();
observation.setStatus(Observation.ObservationStatus.FINAL);
observation.setSubject(new Reference("Patient/" + patient.getIdElement().getIdPart()));
observation.addCategory().addCoding().setSystem("http://terminology.hl7.org/CodeSystem/observation-category").setCode("vital-signs");
observation.setCode(new CodeableConcept().addCoding(new Coding("http://loinc.org", "85354-9", "Vital Signs")));
observation.addComponent(new Observation.ObservationComponentComponent()
                                        .setCode(new CodeableConcept()
                                        .addCoding(new Coding("http://loinc.org", "8310-5", "Body temperature")))
                                        .setValue(new Quantity().setValue(96.8).setUnit("F")));
observation.addComponent(new Observation.ObservationComponentComponent()
                                        .setCode(new CodeableConcept()
                                        .addCoding(new Coding("http://loinc.org", "8480-6", "Systolic blood pressure")))
                                        .setValue(new Quantity().setValue(130).setUnit("mmHg")));
observation.addComponent(new Observation.ObservationComponentComponent()
                                        .setCode(new CodeableConcept()
                                        .addCoding(new Coding("http://loinc.org", "8462-4", "Diastolic blood pressure")))
                                        .setValue(new Quantity().setValue(80).setUnit("mmHg")));

observation.setEncounter(new Reference("Encounter/" + encounterId));
observation.setEffective(new DateTimeType("2020-01-01T00:00:00-05:00"));

//print the observation
System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(observation));   
